In [ ]:
if instanceId is None:
    instanceId = ''
instance_id = instanceId
school_year = schoolYear
api_url = apiUrl

In [ ]:
import requests

class EdFiApiManager:
    def __init__(self, api_url, instance_id, school_year):
        self.api_url = api_url
        self.instance_id = instance_id
        self.school_year = school_year
        self.api_version = api_url.split('/')[-3]
        self.base_response = self.fetch_json_from_api(api_url)
        self.urls = self.base_response['urls']

    def fetch_json_from_api(self, api_url):
        try:
            response = requests.get(api_url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

    def replace_url_component(self, url, index, new_component, num_parts):
        url_parts = url.split('/')
        if len(url_parts) == num_parts:
            url_parts[index] = new_component
            return "/".join(url_parts)
        else:
            return url

    def update_urls(self):
        for key, url in self.urls.items():
            url = url.replace('{instance}', self.instance_id)
            self.urls[key] = url

        self.dependencies_url = self.replace_url_component(self.urls['dependencies'], -2, self.school_year, 12)
        self.openapi_metadata_url = self.replace_url_component(self.urls['openApiMetadata'], -1, self.school_year, 9)
        self.data_management_url = self.replace_url_component(self.urls['dataManagementApi'], -1, self.school_year, 10)
        self.auth_url = self.urls['oauth']
    
    def set_other_metadata(self):
        self.other_metadata = self.fetch_json_from_api(self.openapi_metadata_url)
    
    def get_referenced_url(self,url_type):
        other_metadata = self.other_metadata
        for item in other_metadata:
            if item['name'] == url_type:
                return item['endpointUri']
        return None

In [ ]:
edfi_api_manager = EdFiApiManager(api_url, instance_id, school_year)
edfi_api_manager.update_urls()
edfi_api_manager.set_other_metadata()

In [ ]:
dependenciesUrl = edfi_api_manager.dependencies_url
openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
dataManagementUrl = edfi_api_manager.data_management_url
authUrl = edfi_api_manager.auth_url

changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

apiVersion = edfi_api_manager.api_version
apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion